In [1]:
import autokeras as ak

In [2]:
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers

In [3]:
tf.keras.backend.set_image_data_format('channels_last')

In [4]:
SEED = 1
BATCH_SIZE = 10
DATA_ROOT = Path('/', 'data', '02-tfrecords')
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[1], True)

In [6]:
features = {
    'label': tf.io.FixedLenFeature([], dtype=tf.int64),
    'image': tf.io.FixedLenFeature([], dtype=tf.string),
}

def parse_example(example_proto):
    return tf.io.parse_example(example_proto, features)

def parse_record(example):
    image = tf.io.decode_raw(example['image'], tf.uint8)
    image = tf.reshape(image, (-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = example['label']

    return image, label  

In [7]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

def prepare_dataset(paths, batch_size, augment=False, seed=SEED):
    ds = tf.data.TFRecordDataset(paths, num_parallel_reads=tf.data.AUTOTUNE, compression_type='GZIP')
    ds = ds.shuffle(10000, seed=seed, reshuffle_each_iteration=True)
    ds = ds.map(parse_example)
    ds = ds.batch(batch_size)        
    ds = ds.map(parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    if augment:    
        ds = ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.repeat(2)

    return ds.prefetch(tf.data.AUTOTUNE)

In [8]:
train_paths = [str(p) for p in Path(DATA_ROOT, 'train').glob('*.tfrecord')]
train_dataset = prepare_dataset(train_paths, BATCH_SIZE, augment=True)

test_paths = [str(p) for p in Path(DATA_ROOT, 'test').glob('*.tfrecord')]
test_dataset = prepare_dataset(test_paths, BATCH_SIZE)

In [9]:
auto_model = ak.ImageClassifier(
    num_classes=3,
    max_trials=30,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    seed=SEED,
    tuner='random',
    max_model_size=50000000,
    directory='/automl',
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reloading Oracle from existing project ./image_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./image_classifier/tuner0.json


In [ ]:
auto_model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=25,
)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu